In [ ]:
from __future__ import division, print_function
from gensim import models
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Dropout, Reshape, Flatten, concatenate, Input, Conv1D, GlobalMaxPooling1D, Embedding
from keras.layers import LSTM
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Model
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import os
import collections
import re
import string

In [ ]:
data = pd.read_csv('new_train.csv')

In [ ]:
#data.Label.unique()
data.emotion.unique()

array(['sadness', 'anger', 'love', 'surprise', 'fear', 'joy'],
      dtype=object)

In [ ]:
data.shape

(16000, 2)

In [ ]:
sadness = []
anger = []
love= []
surprise = []
fear = []
joy = []
for l in data.emotion:
    if l == "sadness":
        sadness.append(1)
        anger.append(0)
        love.append(0)
        surprise.append(0)
        fear.append(0)
        joy.append(0)
    elif l == "anger":
        sadness.append(0)
        anger.append(1)
        love.append(0)
        surprise.append(0)
        fear.append(0)
        joy.append(0)
    elif l=="love":
        sadness.append(0)
        anger.append(0)
        love.append(1)
        surprise.append(0)
        fear.append(0)
        joy.append(0)
    elif l == "surprise":
        sadness.append(0)
        anger.append(0)
        love.append(0)
        surprise.append(1)
        fear.append(0)
        joy.append(0)
    elif l == "fear":
        sadness.append(0)
        anger.append(0)
        love.append(0)
        surprise.append(0)
        fear.append(1)
        joy.append(0)
    elif l == "joy":
        sadness.append(0)
        anger.append(0)
        love.append(0)
        surprise.append(0)
        fear.append(0)
        joy.append(1)

In [ ]:
data['sadness']= sadness
data['anger']= anger
data['love'] = love
data['surprise'] = surprise
data['fear'] = fear
data['joy'] = joy

In [ ]:
data.head()

text  emotion  sadness  anger  \
0                            i didnt feel humiliated  sadness        1      0   
1  i can go from feeling so hopeless to so damned...  sadness        1      0   
2   im grabbing a minute to post i feel greedy wrong    anger        0      1   
3  i am ever feeling nostalgic about the fireplac...     love        0      0   
4                               i am feeling grouchy    anger        0      1   

   love  surprise  fear  joy  
0     0         0     0    0  
1     0         0     0    0  
2     0         0     0    0  
3     1         0     0    0  
4     0         0     0    0

CLEAN

In [ ]:
def remove_punct(text):
    text_nopunct = ''
    text_nopunct = re.sub('['+string.punctuation+']', '', text)
    return text_nopunct

data['Text_Clean'] = data['text'].apply(lambda x: remove_punct(x))

In [ ]:
import nltk
nltk.download('punkt')
from nltk import word_tokenize, WordNetLemmatizer
tokens = [word_tokenize(sen) for sen in data.Text_Clean] 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
def lower_token(tokens): 
    return [w.lower() for w in tokens]    
    
lower_tokens = [lower_token(token) for token in tokens] 

In [ ]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stoplist = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def remove_stop_words(tokens): 
    return [word for word in tokens if word not in stoplist]

In [ ]:
filtered_words = [remove_stop_words(sen) for sen in lower_tokens] 

In [ ]:
result = [' '.join(sen) for sen in filtered_words] 
data['Text_Final'] = result

In [ ]:
data['tokens'] = filtered_words

In [ ]:
data = data[['Text_Final', 'tokens', 'emotion', 'sadness',	'anger',	'love',	'surprise',	'fear','joy']]

In [ ]:
data[:4]

Text_Final  \
0                              didnt feel humiliated   
1  go feeling hopeless damned hopeful around some...   
2          im grabbing minute post feel greedy wrong   
3  ever feeling nostalgic fireplace know still pr...   

                                              tokens  emotion  sadness  anger  \
0                          [didnt, feel, humiliated]  sadness        1      0   
1  [go, feeling, hopeless, damned, hopeful, aroun...  sadness        1      0   
2  [im, grabbing, minute, post, feel, greedy, wrong]    anger        0      1   
3  [ever, feeling, nostalgic, fireplace, know, st...     love        0      0   

   love  surprise  fear  joy  
0     0         0     0    0  
1     0         0     0    0  
2     0         0     0    0  
3     1         0     0    0

In [ ]:
data_train, data_test = train_test_split(data, test_size=0.10, random_state=42)

In [ ]:
all_training_words = [word for tokens in data_train["tokens"] for word in tokens]
training_sentence_lengths = [len(tokens) for tokens in data_train["tokens"]]
TRAINING_VOCAB = sorted(list(set(all_training_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_training_words), len(TRAINING_VOCAB)))
print("Max sentence length is %s" % max(training_sentence_lengths))

134764 words total, with a vocabulary size of 14195
Max sentence length is 35


In [ ]:
all_test_words = [word for tokens in data_test["tokens"] for word in tokens]
test_sentence_lengths = [len(tokens) for tokens in data_test["tokens"]]
TEST_VOCAB = sorted(list(set(all_test_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_test_words), len(TEST_VOCAB)))
print("Max sentence length is %s" % max(test_sentence_lengths))

14873 words total, with a vocabulary size of 4177
Max sentence length is 35


In [ ]:
!wget nlp.stanford.edu/data/glove.6B.zip

--2022-12-14 22:11:08--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2022-12-14 22:11:08--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-12-14 22:11:09--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [ ]:
! unzip glove*.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [ ]:
import gensim

In [ ]:
from gensim.scripts.glove2word2vec import glove2word2vec

In [ ]:
glove_input_file = 'glove.6B.300d.txt'

In [ ]:
glove_output_file = 'word2vec.txt'

In [ ]:
glove2word2vec(glove_input_file, glove_output_file)

(400000, 300)

In [ ]:
from gensim.models import KeyedVectors

In [ ]:
word2vec = KeyedVectors.load_word2vec_format(glove_output_file, binary = False)

In [ ]:
def get_average_word2vec(tokens_list, vector, generate_missing=False, k=300):
    if len(tokens_list)<1:
        return np.zeros(k)
    if generate_missing:
        vectorized = [vector[word] if word in vector else np.random.rand(k) for word in tokens_list]
    else:
        vectorized = [vector[word] if word in vector else np.zeros(k) for word in tokens_list]
    length = len(vectorized)
    summed = np.sum(vectorized, axis=0)
    averaged = np.divide(summed, length)
    return averaged

def get_word2vec_embeddings(vectors, clean_comments, generate_missing=False):
    embeddings = clean_comments['tokens'].apply(lambda x: get_average_word2vec(x, vectors, 
                                                                                generate_missing=generate_missing))
    return list(embeddings)

In [ ]:
training_embeddings = get_word2vec_embeddings(word2vec, data_train, generate_missing=True)

In [ ]:
MAX_SEQUENCE_LENGTH = 50
EMBEDDING_DIM = 300

In [ ]:
tokenizer = Tokenizer(num_words=len(TRAINING_VOCAB), lower=True, char_level=False)
tokenizer.fit_on_texts(data_train["Text_Final"].tolist())
training_sequences = tokenizer.texts_to_sequences(data_train["Text_Final"].tolist())

train_word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(train_word_index))

Found 14195 unique tokens.


In [ ]:
train_cnn_data = pad_sequences(training_sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [ ]:
train_embedding_weights = np.zeros((len(train_word_index)+1, EMBEDDING_DIM))
for word,index in train_word_index.items():
    train_embedding_weights[index,:] = word2vec[word] if word in word2vec else np.random.rand(EMBEDDING_DIM)
print(train_embedding_weights.shape)

(14196, 300)


In [ ]:
test_sequences = tokenizer.texts_to_sequences(data_test["Text_Final"].tolist())
test_cnn_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)
test_cnn_data.shape

(1600, 50)

In [ ]:
def ConvNet(embeddings, max_sequence_length, num_words, embedding_dim, labels_index):
    
    embedding_layer = Embedding(num_words,
                            embedding_dim,
                            weights=[embeddings],
                            input_length=max_sequence_length,
                            trainable=False)
    
    sequence_input = Input(shape=(max_sequence_length,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)

    convs = []
    filter_sizes = [2,3,4,5,6]

    for filter_size in filter_sizes:
        l_conv = Conv1D(filters=200, kernel_size=filter_size, activation='relu')(embedded_sequences)
        l_pool = GlobalMaxPooling1D()(l_conv)
        convs.append(l_pool)


    l_merge = concatenate(convs, axis=1)

    x = Dropout(0.1)(l_merge)  
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)
    preds = Dense(labels_index, activation='sigmoid')(x)

    model = Model(sequence_input, preds)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['acc'])
    model.summary()
    return model


In [ ]:
label_names = ['sadness',	'anger',	'love',	'surprise',	'fear',	'joy']

In [ ]:
y_train = data_train[label_names].values

In [ ]:
x_train = train_cnn_data
y_tr = y_train

In [ ]:
model = ConvNet(train_embedding_weights, MAX_SEQUENCE_LENGTH, len(train_word_index)+1, EMBEDDING_DIM, 
                len(list(label_names)))

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 50)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 50, 300)      4258800     ['input_1[0][0]']                
                                                                                                  
 conv1d (Conv1D)                (None, 49, 200)      120200      ['embedding[0][0]']              
                                                                                                  
 conv1d_1 (Conv1D)              (None, 48, 200)      180200      ['embedding[0][0]']              
                                                                                              

In [ ]:
num_epochs = 3
batch_size = 34

In [ ]:
hist = model.fit(x_train, y_tr, epochs=num_epochs, validation_split=0.1, shuffle=True, batch_size=batch_size)

Epoch 1/3
382/382 [==============================] - 62s 160ms/step - loss: 0.2432 - acc: 0.6888 - val_loss: 0.1393 - val_acc: 0.8222
Epoch 2/3
382/382 [==============================] - 62s 163ms/step - loss: 0.1043 - acc: 0.8859 - val_loss: 0.1078 - val_acc: 0.8674
Epoch 3/3
382/382 [==============================] - 61s 159ms/step - loss: 0.0561 - acc: 0.9459 - val_loss: 0.1052 - val_acc: 0.8764


In [ ]:
predictions = model.predict(test_cnn_data, batch_size=1024, verbose=1)

2/2 [==============================] - 3s 969ms/step


In [ ]:
test_cnn_data.shape

(1600, 50)

In [ ]:
prediction_labels=[]
for p in predictions:
    prediction_labels.append(label_names[np.argmax(p)])
print((prediction_labels))

['sadness', 'joy', 'sadness', 'joy', 'sadness', 'sadness', 'fear', 'sadness', 'love', 'fear', 'anger', 'anger', 'joy', 'sadness', 'sadness', 'surprise', 'anger', 'joy', 'love', 'surprise', 'anger', 'joy', 'sadness', 'sadness', 'sadness', 'love', 'love', 'sadness', 'sadness', 'joy', 'sadness', 'joy', 'love', 'joy', 'love', 'sadness', 'sadness', 'fear', 'joy', 'joy', 'joy', 'anger', 'joy', 'sadness', 'anger', 'sadness', 'love', 'joy', 'sadness', 'sadness', 'anger', 'fear', 'joy', 'love', 'sadness', 'anger', 'sadness', 'joy', 'joy', 'joy', 'fear', 'joy', 'joy', 'joy', 'anger', 'joy', 'sadness', 'anger', 'sadness', 'anger', 'fear', 'sadness', 'anger', 'fear', 'sadness', 'love', 'joy', 'joy', 'anger', 'anger', 'fear', 'joy', 'anger', 'sadness', 'surprise', 'fear', 'fear', 'sadness', 'sadness', 'love', 'love', 'fear', 'sadness', 'love', 'sadness', 'sadness', 'fear', 'joy', 'love', 'joy', 'joy', 'love', 'sadness', 'joy', 'anger', 'joy', 'joy', 'sadness', 'joy', 'sadness', 'sadness', 'sadness'

In [ ]:
accuracy =sum(data_test.emotion==prediction_labels)/len(prediction_labels)
print("Accuracy: ")
print(accuracy)

Accuracy: 
0.894375
